In [1]:
import sys
sys.path.append('../')

In [2]:
from Datasets.BaseballDataset import BaseballDataset

In [3]:
import pandas as pd

data = pd.read_csv("../data/statcast_mini_cleaned.csv")


In [4]:
train_mini = data[0:70000]
valid_mini = data[70000:].reset_index().drop(columns=['index'])

In [8]:
train_mini

,launch_speed,game_date,release_speed,release_pos_x,release_pos_z,batter,pitcher,balls,strikes,pfx_x,...,hit_location_2.0,hit_location_3.0,hit_location_4.0,hit_location_5.0,hit_location_6.0,hit_location_7.0,hit_location_8.0,hit_location_9.0,outs_when_up_1,outs_when_up_2
0,-0.627173,2024-06-30,-1.056521,-1.498172,-2.802440,680869,623149,0.121914,1.339073,1.591409,...,True,False,False,False,False,False,False,False,False,True
1,1.645207,2024-06-30,-1.089609,-1.535090,-2.936805,680869,623149,0.121914,1.339073,1.568781,...,False,False,False,False,False,False,False,False,False,True
2,1.081636,2024-06-30,0.333173,-1.455979,-2.572100,680869,623149,0.121914,1.339073,-0.773192,...,False,False,False,False,False,False,False,False,False,True
3,-0.627173,2024-06-30,-0.891082,-1.503446,-2.840830,680869,623149,-0.912177,1.339073,1.783745,...,False,False,False,False,False,False,False,False,False,True
4,1.668473,2024-06-30,0.184277,-1.445430,-2.744855,680869,623149,-0.912177,0.129561,-0.626111,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,-0.627173,2024-06-12,0.002293,-0.896922,-0.499034,623993,641729,-0.912177,0.129561,-1.271002,...,False,False,False,False,False,False,False,False,True,False
69996,1.609014,2024-06-12,-1.023433,-0.976034,-0.479839,623993,641729,-0.912177,-1.079951,0.652357,...,False,False,False,False,False,False,False,False,True,False
69997,2.167415,2024-06-12,0.862580,-0.949664,-0.307084,681297,641729,-0.912177,-1.079951,-0.603484,...,False,False,False,False,False,False,False,False,True,False
69998,1.916652,2024-06-12,-0.709098,-1.044598,-0.633400,663624,641729,-0.912177,1.339073,0.856007,...,False,False,False,False,True,False,False,False,True,False


In [9]:
valid_mini

,launch_speed,game_date,release_speed,release_pos_x,release_pos_z,batter,pitcher,balls,strikes,pfx_x,...,hit_location_2.0,hit_location_3.0,hit_location_4.0,hit_location_5.0,hit_location_6.0,hit_location_7.0,hit_location_8.0,hit_location_9.0,outs_when_up_1,outs_when_up_2
0,-0.627173,2024-06-12,0.945300,-0.886374,-0.364669,663624,641729,-0.912177,-1.079951,-0.388520,...,False,False,False,False,False,False,False,False,True,False
1,-0.627173,2024-06-12,0.945300,-0.923293,-0.595010,668939,641729,1.156005,1.339073,-0.637425,...,True,False,False,False,False,False,False,False,False,False
2,1.115243,2024-06-12,-1.155785,-1.018227,-0.652595,668939,641729,1.156005,0.129561,0.720240,...,False,False,False,False,False,False,False,False,False,False
3,-0.627173,2024-06-12,-0.692554,-0.976034,-0.556620,668939,641729,0.121914,0.129561,0.776810,...,False,False,False,False,False,False,False,False,False,False
4,-0.627173,2024-06-12,-1.106153,-1.055146,-0.729375,668939,641729,0.121914,-1.079951,0.776810,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1.409955,2024-06-05,1.110740,-0.169095,0.691058,643396,544150,-0.912177,1.339073,-0.569542,...,False,False,False,False,False,False,False,False,False,True
29996,2.035570,2024-06-05,-0.527114,-0.438075,0.115207,643396,544150,-0.912177,0.129561,0.290313,...,False,False,False,False,False,False,False,False,False,True
29997,1.099732,2024-06-05,0.994932,-0.332592,0.287962,643396,544150,-0.912177,-1.079951,-0.535600,...,False,False,False,False,False,False,False,False,False,True
29998,1.877874,2024-06-05,0.961844,-0.422252,0.230377,672386,544150,2.190096,1.339073,-0.614797,...,False,False,False,False,False,False,True,False,True,False


In [5]:
config = "../data/config.json"
sequence_length = 20 



train_dataset = BaseballDataset(train_mini,config,sequence_length)
valid_dataset = BaseballDataset(valid_mini,config,sequence_length)


c:\Users\declan\Documents\BaseballModeling\Models\..\Datasets\BaseballDataset.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[mask_column] = 0
c:\Users\declan\Documents\BaseballModeling\Models\..\Datasets\BaseballDataset.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[mask_column] = 0


In [6]:
print(len(train_dataset), len(valid_dataset))

61281 22160


In [7]:
train_dataset[0][0].shape

torch.Size([20, 65])

In [8]:
train_dataset[0][1].shape

torch.Size([4])

In [9]:
valid_dataset[0][2]

[tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import torch.nn.functional as F


class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Tensor, shape ``[batch_size, seq_len, embedding_dim]``
        """
        x = x + self.pe[:x.size(1)].transpose(0, 1)
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_heads, num_encoder_layers, hidden_dim, output_dim, sequence_length, dropout=0.1):
        super(TransformerModel, self).__init__()
        
        self.input_dim = input_dim
        self.sequence_length = sequence_length
        
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.positional_encoding = PositionalEncoding(hidden_dim, dropout)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, dim_feedforward=hidden_dim, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_encoder_layers)
        
        self.fc_layers = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.embedding(x)
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)
        x = x[:, -1, :]  # Use the output of the last pitch in the sequence
        x = self.fc_layers(x)
        return x

class CustomLoss(nn.Module):
    def __init__(self, weight_param):
        super(CustomLoss, self).__init__()
        self.weight_param = weight_param

    def forward(self, output, target_continuous, target_categorical):
        # Continuous target loss (MSE)
        mse_loss = F.mse_loss(output[:, :target_continuous.size(1)], target_continuous)
        
        # Categorical target loss (Cross-Entropy) for each categorical feature
        cross_entropy_loss = 0
        start_idx = target_continuous.size(1)
        for cat_target in target_categorical:
            end_idx = start_idx + cat_target.size(1)
            cross_entropy_loss += F.cross_entropy(output[:, start_idx:end_idx], cat_target.argmax(dim=1))
            start_idx = end_idx
        
        # # Sum of all categorical losses
        # cross_entropy_loss = torch.sum(categorical_losses)

        # Weighted sum of the losses
        loss = (self.weight_param * mse_loss) + ((1 - self.weight_param) * cross_entropy_loss)
        return loss

# # Hyperparameters
# input_dim = 65 # Number of features in a single pitch
# num_heads = 4
# num_encoder_layers = 4
# hidden_dim = 40 # Increased hidden dimension for better representation
# output_dim = 6  # Number of label dimensions
# sequence_length = 20
# dropout = 0.1
# batch_size = 32

# # Initialize the model, loss function, and optimizer
# model = TransformerModel(input_dim, num_heads, num_encoder_layers, hidden_dim, output_dim, sequence_length, dropout)
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Dummy data for testing
# dummy_input = torch.randn(batch_size, sequence_length, input_dim)
# dummy_target_continuous = torch.randn(batch_size, 3)  # Assuming 3 continuous targets
# dummy_target_categorical = torch.randint(0, 2, (batch_size, 3))  # Assuming 3 categorical targets

# # Forward pass
# output = model(dummy_input)

# # Define the custom loss function
# weight_param = 0.5  # Adjust this weight parameter as needed
# criterion = CustomLoss(weight_param)

# # Compute the loss
# loss = criterion(output, dummy_target_continuous, dummy_target_categorical)
# print(f'Loss: {loss.item()}')


In [3]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

44606

In [12]:
from torch.utils.data import DataLoader

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [16]:

input_dim = 65  # Number of features in a single pitch
num_heads = 4
num_encoder_layers = 4
hidden_dim = 32
output_dim = 24 # Number of label dimensions
sequence_length = 20
dropout = 0.1

# Initialize the model, loss function, and optimizer
model = TransformerModel(input_dim, num_heads, num_encoder_layers, hidden_dim, output_dim, sequence_length, dropout)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CustomLoss(weight_param=0.5)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for sequences, cont_targets, cat_targets in train_loader:
        sequences, cont_targets = sequences.to(device), cont_targets.to(device)
        cat_targets = [t.to(device) for t in cat_targets]
        
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, cont_targets, cat_targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    return epoch_loss

def evaluate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for sequences, cont_targets, cat_targets in val_loader:
            sequences, cont_targets = sequences.to(device), cont_targets.to(device)
            cat_targets = [t.to(device) for t in cat_targets]
            
            outputs = model(sequences)
            loss = criterion(outputs, cont_targets, cat_targets)
            
            running_loss += loss.item()
    
    epoch_loss = running_loss / len(val_loader)
    return epoch_loss

num_epochs = 2

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss = evaluate(model, val_loader, criterion, device)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

cuda
Epoch 1/2, Train Loss: 0.7876, Val Loss: 0.6165
Epoch 2/2, Train Loss: 0.6511, Val Loss: 0.6019
